In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Set cố định random seed
np.random.seed(42)
tf.random.set_seed(42)

# Tìm hiểu dữ liệu

In [ ]:
pos_df = pd.read_csv('../input/vn-reviews-data/positive_data.csv')
pos_df.head(5)

In [ ]:
neg_df = pd.read_csv('../input/vn-reviews-data/negative_data.csv')
neg_df.head(5)

Hãy đếm số giá trị positive và negative?

In [ ]:
pos_df['Label'].value_counts()

In [ ]:
neg_df["Label"].value_counts()

Trong một số trường hợp, số lượng samples của các class có tỉ lệ quá khác biệt sẽ ảnh hưởng đến chất lượng của mô hình.
Vì vậy nên sử dụng phương thức chọn mẫu `DataFrame.sample()` để cho số lượng mẫu positive bằng với số lượng mẫu negative.

In [ ]:
pos_df_sampled = pos_df.sample(n=len(neg_df), random_state=42)

Kết hợp dữ liệu positive và negative vào cùng một DataFrame bằng `pandas.concat`

In [ ]:
df = pandas.concat([pos_df_sampled, neg_df])
df['Label'].value_counts()

In [ ]:
# Vì sử dụng phương thức pandas.concat() nên các sample positive và negative hiện đang không được phân bố đồng đều trong DataFrame
df.tail(10)

Để phân phối lại các sample positive và negative trong df, sử dụng phương thức [DataFrame.sample](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html).

Ở đây ta chọn `frac=1` để chọn **tất cả** các sample trong df với thứ tự ngẫu nhiên.

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

Đầu ra của mô hình phân loại hai lớp (binary classification) là giá trị 0 hoặc 1.

Vì vậy cần chuyển các giá trị -1 thành 0 bằng [DataFrame.replace](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html).

In [ ]:
# DataFrame.replace() nhận vào một dictionary
# với key là giá trị cần thay đổi và value là giá trị mới
df['Label'] = df['Label'].replace({-1:0})
df['Label'].value_counts()

# Chia tập train - test

Sử dụng thuộc tính `values` của Pandas Series để lấy các giá trị dưới dạng **numpy ndarray**.

Các dữ liệu dùng để train và test nên được chuyển đổi sang kiểu `ndarray` để phù hợp với các hàm tính toán trong Machine Learning và Deep Learning.

In [ ]:
X = df['Review'].values
y = df['Label'].values
X.shape, y.shape

Sử dụng hàm `train_test_split` của sklearn để chia tập train và test theo tỉ lệ 8:2.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train.shape, X_test.shape

# Tokenize dữ liệu train - test
Vận dụng các kiến thức tokenizing và padding ở buổi trước để xử lý dữ liệu chữ.\
Nhắc lại các bước xử lý:
1. Chọn kích thước từ điển
2. Chọn độ dài lớn nhất của một chuỗi (sequence)
3. Tạo Tokenizer và fit Tokenizer (trên văn bản của tập train)
4. Sử dụng Tokenizer đã huấn luyện để tạo tokens
5. Sử dụng padding và truncating để các chuỗi có độ dài bằng nhau

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

In [ ]:
# Chọn kích thước từ điển là 10000
# và độ dài lớn nhất của một bình luận là 400 tokens
vocab_size = 10000
max_length = 400

# Tạo tokenizer và fit trên tập train
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Sử dụng tokenizer đã fit để tokenize các bình luận trên tập train
X_train_seqs = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_seqs, maxlen=max_length, padding='post', truncating='post') # thực hiện padding

# Sử dụng tokenizer đã fit để tokenize các bình luận trên tập test
X_test_seqs = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_seqs, maxlen=max_length, padding='post', truncating='post') # thực hiện padding

print("Length of train dataset:", len(X_train_padded))
print("Length of test dataset:", len(X_test_padded))

# Định nghĩa mô hình NLP

Mô hình phân loại ngôn ngữ đơn giản gồm 2 phần:
1. Layer Embedding: Để học ngôn ngữ
2. Block phân loại: Tương tự với mô hình phân loại hình ảnh

Vì mục tiêu bài toán là phân loại 02 lớp nên cần sử dụng loss là `binary_crossentropy` và metric là `accuracy`.\
Mặc định sử dụng optimizer là `adam`.

In [ ]:
# Chọn chiều embedding là 128
embedding_dim = 128

model = keras.Sequential([
    # Layer Embedding
    keras.layers.Embedding(vocab_size, embedding_dim, trainable=True), # vocab_size=10000, max_length=400
    # Layer phân loại (đã học)
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),                                      
    keras.layers.Dense(1, activation='sigmoid'),
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])       # mô hình phân loại thường sử dụng thang đo là acc
model.summary()

Sử dụng thêm ModelCheckpoint và chạy phương thức `fit` để huấn luyện mô hình.

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint 

checkpoint = ModelCheckpoint(
    filepath='model_epoch_{epoch:02d}.keras',
    save_weights_only=False,
    save_best_only=False,
    monitor='val_loss',
    verbose=1
)

# Huấn luyện mô hình với config đã chọn
history = model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=8,                            # Số lần train toàn bộ dữ liệu
    callbacks=[checkpoint]               # Thêm config checkpoint
)

# Sử dụng mô hình
Hãy tạo một số dữ liệu mẫu để đánh giá mô hình.\
Ở đây ta có một số bình luận mẫu `test_feedback` và giá trị phân loại tương ứng `test_truth`.\

In [ ]:
test_reviews = [
    "Thịt bị hôi. Nước bún thì nhạt. Đề nghị đừng ăn.",
    "Đồ ăn k giống review trên mạng, chất lượng tệ, không ngon. Tốn tiền!",
    "Rất thích không gian quán",
    "Không thể tin được phải đi 5km để ăn một món như thế này",
    "Không thể tin được chỉ cần đi 5km để ăn một món như thế này",
    "Tôi không ghét món dưa leo lắm",
    "Tôi không ghét món dưa leo lắm. Ăn cũng được",
    "Đồ ăn nhiều bột ngọt. Không ăn nữa",
    "Nhiều đồ ăn, khá ngon. Nhưng giá cả hơi mắc. Tuy nhiên không gian quán đẹp, sẽ quay lại lần sau"
    ]
test_gt = [0, 0, 1, 0, 1, 1, 1, 0, 1]
test_df = pd.DataFrame({"Review": test_reviews, "Label": test_gt})
test_df

Các bước sử dụng mô hình để phân loại:
1. Sử dụng tokenizer đã fit ở bước trước để tokenize các bình luận
2. Thực hiện padding tương ứng
3. Dự đoán bằng phương thức `predict` với mô hình
4. Xử lý kết quả dự đoán

In [ ]:
# Sử dụng lại tokenizer
test_reviews = test_df['Review'].values
test_labels = test_df['Label'].values
test_seqs = tokenizer.texts_to_sequences(test_reviews)
test_padded = pad_sequences(test_seqs, padding='post', truncating='post', maxlen=max_length) # thực hiện padding

In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model("model_epoch_04.keras")
preds = loaded_model.predict(test_padded)
preds

In [ ]:
classes = {"1":"positive", "0": "negative"}
acc = 0
i = 0
threshold = 0.5

for fb, score in zip(test_reviews, preds):
    prediction = (score>threshold).astype(int)
    print(fb, "|", classes[str(test_labels[i])], "predicted as", classes[str(prediction[0])], "with score =", score[0])
    
    if prediction == test_labels[i]:
        acc += 1
    i += 1
    
print("\nAccuracy:", round(acc/len(test_reviews), 2))